<img src='images/gdd-logo.png' width='300px' align='right' style="padding: 15px">

# Pandas

Let's start exploring datasets. For this we are going to use the `pandas` library.

In this section we will cover:

- [Importing the library](#import)
- [Exploratory analysis](#exploratory)
- [Data Cleaning](#cleaning)
- [Plotting with `matplotlib`](#plotting)

<a id='import'></a>

## Importing

Pandas is a sepcialised package that allows us to work with databases using python.

First we need to import the package.

In [ ]:
import pandas as pd

<a id='exploratory'></a>

## Exploratory analysis

The dataset  we'll be using was downloaded as a CSV from [this Google Trends query](https://trends.google.com/trends/explore?date=2004-01-01%202020-12-31&q=%2Fm%2F05z1_,%2Fm%2F075st,%2Fm%2F0212jm,%2Fm%2F02p97,%2Fm%2F080_2).

<img src='images/what-i-think-i-do.png' width='600px' align='left'>

filepath: `data/programming-languages.csv`

This is a time series indexed by month with the search interest for each browser.

- **Month:** Each month from 2004-01 to 2020-12
- **Python:** Search interest for 
- **SQL:** Search interest for Python
- **R:** Search interest for R
- **JavaScript:** Search interest for JavaScript
- **Visual Basic for Applications:** Search interest for VBA

Google defines the values of search interest as:

Numbers represent search interest relative to the highest point on the chart for the given region and time. A value of 100 is the peak popularity for the term. A value of 50 means that the term is half as popular. A score of 0 means there was not enough data for this term.

Best of luck and may the best browser win!

In [ ]:
# Create data to use:

trends = pd.read_csv('data/programming-trends.csv')

In [ ]:
# look at the top 5 rows

In [ ]:
# fetch column names


In [ ]:
# get info on all the columns


In [ ]:
# get stats on the numeric columns - where's R and VBA?


In [ ]:
# find the min and max of the data


In [ ]:
# check the data types of each column


### Data Cleaning (wrangling)

<img src='images/spend-time.jpg' width='700px'>

Photo: [Forbes](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/?sh=4cd1339a6f63)

DataFrame methods, like string methods, allow us to alter/change the dataframe without overwriting the original data.

- `.assign()`: Add new columns
- `.set_index()`: Set an index column
- `.replace()`: Replace values
- `.astype()`: Convert the datatypes in the df
- `.rename()`: rename columns or index names

In [ ]:
(
    trends               
#     .assign(Month = pd.to_datetime(trends['Month'])) # convert Month to pd datetime
#     .set_index('Month')   # set Month as the index col
#     .replace('<5','5')    # replace exact values in the df
#     .astype(int)          # change all columns to dtype int
#     .rename(columns={'Visual Basic for Applications':'VBA'}) # rename columns (not values)
).head()

In [ ]:
trends_cleaned = (
    trends               
    .assign(Month = pd.to_datetime(trends['Month'])) # convert Month to pd datetime
    .set_index('Month')   # set Month as the index col
    .replace('<5','5')    # replace exact values in the df
    .astype(int)          # change all columns to dtype int
    .rename(columns={'Visual Basic for Applications':'VBA'}) # rename columns (not values)
)

### Activity:

Using trends_cleaned, find out which language has the smallest mean. Answer the poll when you have your answer.

## In-depth Analysis

Our index is now the month column. This will be really useful because indexes are really special in Python. And datetime indexes are even more special.

In [ ]:
trends_cleaned.index

To access a certain month we can use the `.loc[]` accessor.

Say we want to look at December 2016 (date is formatted as `yyyy-mm-dd`):

In [ ]:
trends_cleaned.loc['2016-12-01']

Now we can compare years! Let's compare the first month of 2004 to the first month of 2020.

In [ ]:
start_year = trends_cleaned.loc['2016-12-01'] 
end_year = trends_cleaned.loc['2016-12-01']

start_year, end_year

We can also do calculations - maybe we want to look at the % change from Jan 2004 - Jan 2020:

In [ ]:
pct_change_all = (end_year-start_year)/start_year
pct_change_all

How about we only look at Python...

In [ ]:
print(f"% change in Python search interest (Jan04 - Jan20): %")

<a id='plotting'></a>
## Plotting

It's time to start visualising this data. Do our hypotheses hold up? Are we surprised? What is the data telling us?

Let's import matplotlib

In [ ]:
# install libraries and set some styling parameters
import matplotlib.pyplot as plt
import matplotlib

# set matplotlib to show charts in the notebook
%matplotlib inline

# set fontsize for charts
font = {'size': 12}
matplotlib.rc('font', **font)

Plot the search interest of each language over time to compare:

Smooth out the lines using rolling average or resample:

In [ ]:
(
    trends_cleaned
    .rolling(window=12)
    .mean()
)

Let's start looking at the percentage change in search interest over time...

Remember our pct_change_all data from earlier?

In [ ]:
# plot the data on the axes: ax
ax = (
    pct_change_all
    .plot(kind='barh',
          title='Percent change in Search Interest from 2004 to 2020 in programming languages')
)

We can use the method `.pct_change()` to look at the percentage change from *n* observations ago. Since every observation is a month let's use the parameter `periods=12` to look at the change each month from the year before.

Let's look at the *Annual Percent Change of Python Search Interest for Each Month*.

In [ ]:
ax = (
    trends_cleaned
    .pct_change(12)
    ['Python']
    .plot()
)

That's a bit hard to read, and comparing with other languages will be very difficult on one plot. Let's use subplots (and a little bit of coloring magic) to compare.

In [ ]:
# look at the percentage increase in each month from 12 months ago (previous year)
trends_pct_change = trends_cleaned.pct_change(12)

# set up 5 vertical subplots
fig, axs = plt.subplots(5, figsize=(15,12))

# loop over each subplot & each column
for ax, col in zip(axs, trends_pct_change.columns):
    # plot the percentage change for each language
    ax.plot(trends_pct_change.index, trends_pct_change[col],  color='k') 
    
    # fill the space between the line and zero with GREEN if positive and RED if negative.
    ax.fill_between(trends_pct_change.index, trends_pct_change[col], where=trends_pct_change[col]<=0, color='r', alpha=0.2)
    ax.fill_between(trends_pct_change.index, trends_pct_change[col], where=trends_pct_change[col]>=0, color='g', alpha=0.2)
    
    # set y-axis and title
    ax.set_ylim([-0.5,0.5])
    ax.yaxis.grid()
    ax.set_ylabel('% Change')
    ax.set_title(f"Change in Search Interest from Previous Year for {col}")
    
plt.tight_layout(pad=3.0)

Let's have a look at the average percent change year on year:

In [ ]:
yearly_trends = (
    trends_cleaned
    .groupby(trends_cleaned.index.year)
    .mean()
    .pct_change()
)
yearly_trends

In [ ]:
import numpy as np

# set up 5 vertical sublots
fig, axs = plt.subplots(5, figsize=(15,12))

# loop through each column to plot on each subplot
for ax, col in zip(axs, yearly_trends.columns):

    # bar plot using color map for pos/neg values
    ax.bar(
           yearly_trends.index, 
           yearly_trends[col], 
           color=(yearly_trends[col] >= 0).map({True: 'g', False: 'r'}), alpha=0.5
          )
    
    # set labels
    ax.set_title(f"{col}")
    ax.yaxis.grid(which='major')
    ax.set_ylim([-0.35, 0.35])
    major_ticks = np.arange(-0.35,0.36,0.35)
    ax.set_yticks(major_ticks)
    
plt.tight_layout(pad=3.0)

<img src='images/panda.gif' width='300px' align='right' style="padding: 20px">

## Final thoughts...

Learning Pandas can be quite frustrating but with practice (and Google!) it becomes straightforward and extremely powerful.

**Want to learn more? Join us on a public course:**
- [Python for Data Analysts](https://godatadriven.com/training/python-for-data-analysts-training/)
- [Certified Python for Data Science](https://godatadriven.com/training/data-science-python-foundation-training/)
- [And more!](https://godatadriven.com/what-we-do/train/#upcoming)

Interested in our other courses? Download our [Training Guide](https://godatadriven.com/topic/training-brochure/)

---
**If you would like to <mark>save this notebook</mark> go to <mark>File -> Download as</mark> and choose the file type you'd like.**

If you have anaconda installed (or plan to get it) download as an `.ipynb`.

If not and you want to save the work you've done, you can choose `.html`.

<img src='images/visit-repo.png' align='left' style='padding:10px'> 

Alternatively you can click Visit repo at the top to navigate to the github repo where you can download everything as a `.zip` file. 


Full solutions can be found in the [solutions folder](solutions/).